<a href="https://colab.research.google.com/github/robgon-art/personal-llama/blob/main/2_Generate_Q%26A_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai llama_index
!gdown 15vLnOyyJBtkjhizR-FqMiILpSD-tYgT3
!unzip robgon_articles_md.zip

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

In [ ]:
import glob
import pandas as pd

def extract_info_from_file(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f.readlines()[:6]]

    info = {
        'filename': filename,
        'title': lines[0].replace('# ', ''),
        'subtitle': lines[1].replace('## ', ''),
        'author': lines[2],
        'date': lines[4].replace('</br>', ''),
        'nickname': lines[4].split(' - ')[0],
        'url': lines[4].split(' - ')[1].replace('</br></br>', '')
    }
    return info

files = glob.glob('/content/robgon_articles_md/*.md')
data_list = [extract_info_from_file(file) for file in files]

df = pd.DataFrame(data_list)
df

,filename,title,subtitle,author,date,nickname,url
0,/content/robgon_articles_md/2023-03-07_using-o...,Using OpenCLIP for Image Search and Automatic ...,How LAION used more data and new ML training t...,Robert A. Gonsalves,OpenClip - https://medium.com/towards-data-sci...,OpenClip,https://medium.com/towards-data-science/using-...
1,/content/robgon_articles_md/2021-12-01_sex-and...,Sex and Drugs and Organic Topic Modeling,Using GPT-J to analyze the lyrics of Rock & Ro...,Robert A. Gonsalves,Organic Topic Modelling - https://medium.com/t...,Organic Topic Modelling,https://medium.com/towards-data-science/sex-an...
2,/content/robgon_articles_md/2021-06-03_ai-meme...,AI-Memer: Using Machine Learning to Create Fun...,How to create new memes using images from Wiki...,Robert A. Gonsalves,AIMemer - https://medium.com/towards-data-scie...,AIMemer,https://medium.com/towards-data-science/ai-mem...
3,/content/robgon_articles_md/2021-03-01_using-o...,Using OpenAI’s CLIP to Search for Design Patents,Using natural language to search and find over...,Robert A. Gonsalves,CLIP Patents - https://medium.com/towards-data...,CLIP Patents,https://medium.com/towards-data-science/using-...
4,/content/robgon_articles_md/2022-06-09_clip-an...,CLIP and PASTE: Using AI to Create Photo Colla...,How to use ML models to extract objects from p...,Robert A. Gonsalves,Clip-n-Paste - https://medium.com/towards-data...,Clip-n-Paste,https://medium.com/towards-data-science/clip-a...
5,/content/robgon_articles_md/2021-09-01_ai-tune...,#Hands-on Tutorials,# AI-Tunes: Creating New Songs with Artificial...,## How I fine-tuned OpenAI’s GPT-3 to generate...,AITunes - https://medium.com/towards-data-scie...,AITunes,https://medium.com/towards-data-science/ai-tun...
6,/content/robgon_articles_md/2022-09-06_using-a...,Using AI to Create New Comic Strips without Wr...,A tutorial on how to use GPT-3 and DALL-E to g...,Robert A. Gonsalves,AI Comics - https://medium.com/towards-data-sc...,AI Comics,https://medium.com/towards-data-science/using-...
7,/content/robgon_articles_md/2021-07-05_magnet-...,#Hands-on Tutorials,# MAGNet: Modern Art Generator using Deep Neur...,"## How I used CLIP, SWAGAN, and a custom genet...",MAGnet - https://medium.com/towards-data-scien...,MAGnet,https://medium.com/towards-data-science/magnet...
8,/content/robgon_articles_md/2021-04-01_ganscap...,#Hands-on Tutorials,# GANscapes: Using AI to Create New Impression...,"## How I trained StyleGAN2 ADA with 5,000 Impr...",Ganscapes - https://medium.com/towards-data-sc...,Ganscapes,https://medium.com/towards-data-science/gansca...
9,/content/robgon_articles_md/2023-01-17_using-c...,Using ChatGPT as a Creative Writing Partner — ...,How the latest language model from OpenAI can ...,Robert A. Gonsalves,ChatGPT Tunes - https://medium.com/towards-dat...,ChatGPT Tunes,https://medium.com/towards-data-science/using-...


In [ ]:
def filename_fn(filename):
  entry = df[df['filename'] == filename].iloc[0].to_dict()
  entry.pop('filename', None)  # Remove the 'filename' key if it exists
  return entry

# load documents
documents = SimpleDirectoryReader("/content/robgon_articles_md", file_metadata=filename_fn).load_data()

In [ ]:
print("Num documents:", len(documents))
print()
for doc in documents[:5]:
  print(doc.metadata)
  print(doc.get_content())
  print("****")

Num documents: 632

{'title': 'MachineRay: Using AI to Create Abstract Art', 'subtitle': 'How I trained a GAN using public domain paintings', 'author': 'Robert A. Gonsalves', 'date': 'MachineRay - https://medium.com/towards-data-science/machineray-using-ai-to-create-abstract-art-39829438076a', 'nickname': 'MachineRay', 'url': 'https://medium.com/towards-data-science/machineray-using-ai-to-create-abstract-art-39829438076a'}


MachineRay: Using AI to Create Abstract Art
Robert A. Gonsalves
Aug 3, 2020
MachineRay - https://medium.com/towards-data-science/machineray-using-ai-to-create-abstract-art-39829438076a
For the past three months, I have been exploring the latest techniques in Artificial Intelligence (AI) and Machine Learning (ML) to create abstract art. During my investigation, I learned that three things are needed to create abstract paintings: (A) source images, (B) an ML model, and (C) a lot of time to train the model on a high-end GPU. Before I discuss my work, let’s take a look

In [ ]:
import random
random.shuffle(documents)
select_documents = documents[:100]
print("Num documents:", len(documents))
print()
for doc in select_documents[:5]:
  print(doc.metadata)
  print(doc.get_content())
  print("****")

Num documents: 632

{'title': 'Writing Songs with GPT-4: Part 2, Chords', 'subtitle': 'How to use the latest large language model from OpenAI to help compose chords for original songs', 'author': 'Robert A. Gonsalves', 'date': 'GPT-4 Chords - https://medium.com/towards-data-science/writing-songs-with-gpt-4-part-2-chords-173cfda0e5a1', 'nickname': 'GPT-4 Chords', 'url': 'https://medium.com/towards-data-science/writing-songs-with-gpt-4-part-2-chords-173cfda0e5a1'}


Source Code
The source code for this project is available on  GitHub .

****
{'title': '#Hands-on Tutorials', 'subtitle': '# MAGNet: Modern Art Generator using Deep Neural Networks', 'author': '## How I used CLIP, SWAGAN, and a custom genetic algorithm to create modern paintings from text descriptions', 'date': 'MAGnet - https://medium.com/towards-data-science/magnet-modern-art-generator-using-deep-neural-networks-57537457bb7', 'nickname': 'MAGnet', 'url': 'https://medium.com/towards-data-science/magnet-modern-art-generator-u

In [ ]:
from llama_index.llms import OpenAI
llm = OpenAI(model="gpt-4")

In [ ]:
prefix = """Ask one question about the reference material and answer it.
Don't use the phrase 'reference material.' Instead, use the article's title or the project's nickname.
In the question don't use the phrase 'the author'. Use 'Robert' or 'Robert A. Gonsalves' instead.
In the answer, refer to the author as 'he/him/his' if his name was used in the question.
Use the format:
q: <question>
a: <answer>"""

for doc in select_documents[:5]:
  prompt ="Reference:\n" + doc.get_metadata_str() + "\n" + doc.get_content() + "\n\n" + prefix
  # print(prompt)
  # print("****")
  response = llm.complete(prompt)
  print(response)
  print("****")

Q: What is the main focus of Robert A. Gonsalves' article "Writing Songs with GPT-4: Part 2, Chords"?
A: In his article, Robert A. Gonsalves discusses how to use the latest large language model from OpenAI, GPT-4, to assist in composing chords for original songs. He provides insights into the capabilities of this AI model in the field of music composition.
****
Q: How does Robert use CLIP to filter images for the MAGnet project?
A: Robert uses CLIP, a model designed by OpenAI, to filter images for the MAGnet project. He uses the image and text encoder models of CLIP to perform semantic searches. These models are trained on a dataset of images with corresponding phrases, with the goal of having the encoded images match the encoded phrases. The image encoder system converts images to embeddings, which are lists of 512 floating-point numbers that capture the images’ general features. The text encoder converts a text phrase to a similar embedding that can be compared to image embeddings fo

In [ ]:
import pandas as pd

# Create an empty DataFrame with the desired columns
df_qa = pd.DataFrame(columns=['doc_metadata', 'doc_content', 'question', 'answer'])

for i, doc in enumerate(select_documents):
    prompt = "Reference\n" + str(doc.metadata) + "\n" + doc.get_content() + "\n\n" + prefix

    # Assuming llm.complete(prompt) returns a string in the format "Q: <question>\nA: <answer>"
    response = llm.complete(prompt)

    # Split the response into question and answer parts
    q, a = response.text.split("\n")

    # Extract the actual question and answer text
    question = q[3:].strip()
    answer = a[3:].strip()

    print("entry:", i)
    print("q:", question)
    print("a:", answer)
    print("****")

    # Create a new DataFrame for the current row and concatenate it to the main DataFrame
    new_row = pd.DataFrame({
        'doc_metadata': [str(doc.metadata)],
        'doc_content': [doc.get_content().strip()],
        'question': [question],
        'answer': [answer]
    })
    df_qa = pd.concat([df_qa, new_row], ignore_index=True)

# Now, df_qa contains the desired data and can be used for further analysis or saved to a file.


entry: 0
q: What is the main objective of Robert A. Gonsalves' project 'GPT-4 Chords'?
a: The main objective of his project 'GPT-4 Chords' is to demonstrate how to use the latest large language model from OpenAI, GPT-4, to help compose chords for original songs.
****
entry: 1
q: How did Robert use CLIP to filter the images for the MAGnet project?
a: Robert used CLIP to filter the images by comparing the embedding from the phrase "modern painting" to the embeddings from the paintings. This allowed him to find the top 10,000 images that best matched the phrase "modern painting". This was done to ensure that the images used for training were good examples of modern painting.
****
entry: 2
q: What are the limitations Robert A. Gonsalves found in using DALL-E for generating artwork for comic strips?
a: Robert found that while DALL-E could generate concept renderings for the comic strips, the images often required cleanup before they could be used in production. He also noted that DALL-E str

In [ ]:
df_qa

,doc_metadata,doc_content,question,answer
0,"{'title': 'Writing Songs with GPT-4: Part 2, C...",Source Code\nThe source code for this project ...,What is the main objective of Robert A. Gonsal...,The main objective of his project 'GPT-4 Chord...
1,"{'title': '#Hands-on Tutorials', 'subtitle': '...",Using CLIP to Filter the Images for Training\n...,How did Robert use CLIP to filter the images f...,Robert used CLIP to filter the images by compa...
2,{'title': 'Using AI to Create New Comic Strips...,"Final Thoughts\nComparing the two systems, I f...",What are the limitations Robert A. Gonsalves f...,Robert found that while DALL-E could generate ...
3,{'title': 'Using AI to Create New Comic Strips...,"Mark Madness\nFor the Mark Madness comic, I us...",How did Robert A. Gonsalves modify the images ...,"Robert cleaned up the images in Photoshop, add..."
4,{'title': 'BIG.art: Using Machine Learning to ...,Source Code\nThe source code for this projec...,What machine learning tools does Robert A. Gon...,He uses GLIDE and BSRGAN to create these high-...
...,...,...,...,...
95,{'title': 'Frost Songs: Using AI to Generate M...,"Background\nFor the last five months, I have b...",What was Robert A. Gonsalves' critique of curr...,Robert A. Gonsalves noted that while AI models...
96,{'title': 'AI-Memer: Using Machine Learning to...,Next Steps\nAlthough the results are pretty go...,What is the next model that the developers beh...,The developers at EleutherAI are planning to b...
97,{'title': 'AI-Memer: Using Machine Learning to...,GPT-3 Da Vinci\nOpenAI’s GPT-3 Da Vinci is cur...,How does Robert A. Gonsalves use GPT-3 Da Vinc...,Robert A. Gonsalves uses GPT-3 Da Vinci by cre...
98,{'title': 'Benford’s Law — A Simple Explanatio...,"Summary\nIn this article, I gave an overview o...",What are the three real datasets that Robert A...,Robert A. Gonsalves used the datasets of city/...


In [ ]:
# df_qa.to_csv('robgon_qa.csv', index=False)
df_qa.to_csv('robgon_qa.csv', index=False, encoding='utf-8-sig')
